### Week 1 Excercise

To demonstrate your familiarity with OpenAI and also Ollama, build a tools that takes a technical question, and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [19]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display
import requests

In [12]:
# constants
MODEL_GPT = 'gpt-4o-mini'
MODEL_OLLAMA = 'llama3.1:latest'
OLLAMA_API_URL = "http://127.0.0.1:11434/v1"

In [3]:
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
llama_api_key = "ollama" # not mandatory

In [4]:
system_prompt = """
    You are a principal software engineer who has world class knowledge in python
"""

question = """
    Please explain what this code does and why:
    yield from {book.get("author") for book in books if book.get("author")}
"""

In [5]:
# OpenAI
openai = OpenAI()

In [12]:
response = openai.chat.completions.create(
    model=MODEL_GPT,
    messages=[
        {"role":"system", "content": system_prompt},
        {"role":"user", "content": question},
    ]
)

answer = response.choices[0].message.content
display(Markdown(answer))

The provided line of code is a Python generator expression that uses `yield from` along with a set comprehension. Let's break it down:

### Components of the Code:

1. **`yield from`**:
   - This statement is used within a generator function. It allows a generator to yield all values from another iterable.
   - Essentially, using `yield from` lets you delegate part of the generator’s operations to another generator or iterable.

2. **Set Comprehension**:
   - The part `{book.get("author") for book in books if book.get("author")}` is a set comprehension.
   - It iterates over a collection (in this case, `books`), and for each `book`, it tries to get the `"author"` key.
   - The condition `if book.get("author")` ensures that only those books which have a truthy value for `"author"` are included in the set.
   - The result of the set comprehension is a set of unique authors from the `books` collection.

### Overall Functionality:

- When you put all of this together, this code snippet creates a generator that yields each unique author from the list of `books`, provided that the author is present and truthy (not `None`, an empty string, etc.).

### Example:

For example, if you have a list of `books` that looks like this:

```python
books = [
    {"title": "Book A", "author": "Author 1"},
    {"title": "Book B", "author": "Author 2"},
    {"title": "Book C", "author": None},
    {"title": "Book D", "author": "Author 1"},
    {"title": "Book E", "author": "Author 3"},
]
```

The set comprehension would create a set like this:

```python
{"Author 1", "Author 2", "Author 3"}
```

When using `yield from` in a generator function, the overall effect is that it will yield each of these unique authors when the generator is iterated over.

### Why Use This Approach?

1. **Uniqueness**: The use of a set comprehension ensures that the authors are unique, so if multiple books have the same author, that author will only be yielded once.
  
2. **Readability and Conciseness**: This syntax is concise and expressive, making it clear that we're collecting unique authors without duplicates.

3. **Deferred Execution**: Using a generator allows you to produce results only as needed, which can be more memory efficient than creating a list of authors if there are many books.

In conclusion, this line is a powerful and efficient way to produce a stream of unique authors from a collection of book records.

In [5]:
requests.get("http://localhost:11434").content

b'Ollama is running'

In [14]:
ollama = OpenAI(base_url=OLLAMA_API_URL, api_key='ollama')

In [21]:
stream = ollama.chat.completions.create(model="llama3.1:latest", messages=[
    {"role": "system", "content": "you are a principal engineer"},
    {"role": "user", "content": "explain and make me understand what is 'yield' keyword do in python?"},
],
stream=True
)

In [22]:
streamed_response = ""
update_handle = display(Markdown(""), display_id=True)

for chunk in stream:
    streamed_response += chunk.choices[0].delta.content or ''
    update_display(Markdown(streamed_response), display_id=update_handle.display_id)

The `yield` keyword! It's one of the most fascinating concepts in Python.

So, as a Principal Engineer, I'd love to dive deep into this topic with you.

**What does `yield` mean?**

In simple terms, `yield` is like saying "stop for now." When used within a function, it allows us to write functions that can remember their state between calls. Think of it as a pause in your execution of the code.

When a function contains a `yield` statement, it's called a **generator** function. This type of function generates its values on-the-fly and remembers where it left off when it returns control to its caller.

Let me write an example to make this clearer:
```python
def infinite_sequence(n):
    num = 0
    while True:
        yield num
        num += n

seq = infinite_sequence(2)
print(next(seq))  # prints: 0
print(next(seq))  # prints: 2
print(next(seq))  # prints: 4
```
Here's what happens:

1. When `infinite_sequence(2)` is called, it creates a generator function that starts executing the code inside.
2. Inside the `while` loop, `yield num` causes the execution to pause and return the current value of `num`.
3. The caller (the next statement) receives this value, which is `0`, and prints it out.
4. However, instead of jumping back into the generator function's code at the top again, we call `next(seq)` with the previous value of `num` (which was still 0).
5. This causes the yield statement to "remember" where it left off (`num = 2`) and return the next number in the sequence (`2`).

Now, this sequence will continue indefinitely until we decide to stop, which is why we can call the generator function an "infinite sequence". Each `next()` call yields the next value, allowing us to traverse through all values without having them materialize beforehand!

**What's the difference with a regular for loop?**

Here's another example that demonstrates what yield provides:

```python
for i in range(10):
    print(i)

# Regular way (without yield)
```

In contrast, using `yield` allows us to implement an iterator in a more declarative and memory-efficient manner:
```python
import itertools

def countdown(n):
    n = 9 # example input number
    while n >= 0:
        yield n
        n -= 1

# Let the iterator go infinite if we pass it a large number for simplicity (example value)
seq2 = countdown(10000000)  

# Now calling next() manually returns the first item and decrements our internal state...
print(next(seq2)) # prints:0 with state as required
```
Here, `countdown` produces the values like an iterator should. In both examples above, `yield` is useful when creating functions that produce data flow.

Remember that you might have to iterate over each yield, either directly or indirectly via iterators using tools like Python's built-in loop `for`.

**So why does this make Python "special"?**

This combination of properties allows writing and structuring a whole class of "iterative" methods in the code we write.